### **Training, save with float32**

Do not use operations not supported by k210, https://github.com/kendryte/nncase/blob/v0.2.0-beta4/docs/tflite_ops.md

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

# ─── Config ────────────────────────────────────────────────────────────────
INPUT_SIZE   = (120, 160)
BATCH_SIZE   = 16
NUM_CLASSES  = 7
HEAD_EPOCHS  = 5
FT_EPOCHS    = 15
TOTAL_EPOCHS = HEAD_EPOCHS + FT_EPOCHS
BASE_LR      = 1e-3
FT_LR        = 1e-4
DATA_DIR     = "./data/preprocessedv3"

# ─── Data Generators ──────────────────────────────────────────────────────
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=4,
    width_shift_range=0.08,
    height_shift_range=0.08,
    horizontal_flip=True,
    validation_split=0.2
)
train_gen = datagen.flow_from_directory(
    DATA_DIR, target_size=INPUT_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training', shuffle=True
)
val_gen = datagen.flow_from_directory(
    DATA_DIR, target_size=INPUT_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation', shuffle=False
)

# ─── Compute class weights (if imbalanced) ────────────────────────────────
labels = train_gen.classes
cw = compute_class_weight(class_weight='balanced',
                          classes=np.unique(labels),
                          y=labels)
class_weights = dict(enumerate(cw))

# ─── 1) Load MobileNetV2 backbone ──────────────────────────────────────────
base = MobileNetV2(
    weights='imagenet',
    include_top=False,
    alpha=0.5,
    input_shape=(*INPUT_SIZE, 3)
)

# ─── 2) Attach your custom head ────────────────────────────────────────────
x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(base.input, outputs)

# ─── 3) Phase 1: Train only the head ───────────────────────────────────────
for layer in base.layers:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=BASE_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1),
    ModelCheckpoint('best_head_v3.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]
model.fit(
    train_gen,
    epochs=HEAD_EPOCHS,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=callbacks
)

# ─── 4) Phase 2: Unfreeze last 20 layers & fine-tune ──────────────────────
for layer in base.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FT_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1),
    ModelCheckpoint('best_finetune_v3.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]
model.fit(
    train_gen,
    initial_epoch=HEAD_EPOCHS,
    epochs=TOTAL_EPOCHS,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=callbacks
)

# ─── 5) Save the final float32 Keras model ────────────────────────────────
model.save('road_classifier_v3.h5')

# ─── 6) Convert to FLOAT32 TFLite (no quantization ops) ──────────────────
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# **Do not** set optimizations or integer types
tflite_model = converter.convert()
with open('road_classifier_v3.tflite', 'wb') as f:
    f.write(tflite_model)

print("Saved FLOAT32 TFLite as 'road_classifier_v3.tflite'")











# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import matplotlib.pyplot as plt

# # Configure constants
# INPUT_SIZE_X = 120
# INPUT_SIZE_Y = 160
# BATCH_SIZE = 16
# NUM_CLASSES = 7
# EPOCHS = 20

# # Load base model with reduced size
# base_model = MobileNetV2(
#     weights='imagenet',
#     include_top=False,
#     input_shape=(INPUT_SIZE_Y, INPUT_SIZE_X, 3),
#     alpha=0.35
# )

# # Add custom head with fewer parameters
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(64, activation='relu')(x)  # Further reduced from 64
# predictions = Dense(NUM_CLASSES, activation='softmax')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# # Freeze base layers
# for layer in base_model.layers:
#     layer.trainable = False

# # Data generators with smaller image size
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     zoom_range=0.2,
#     validation_split=0.2  # Create validation split from training data
# )

# train_generator = train_datagen.flow_from_directory(
#     './data/preprocessedv3',
#     target_size=(INPUT_SIZE_Y, INPUT_SIZE_X),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     shuffle=True,
#     subset='training'  # Set as training data
# )

# validation_generator = train_datagen.flow_from_directory(
#     './data/preprocessedv3',
#     target_size=(INPUT_SIZE_Y, INPUT_SIZE_X),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     shuffle=False,
#     subset='validation'  # Set as validation data
# )

# # Compile & fit
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# history = model.fit(
#     train_generator,
#     epochs=EPOCHS,
#     validation_data=validation_generator
# )

# # Plot training history
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['accuracy'], label='Train Acc')
# plt.plot(history.history['val_accuracy'], label='Val Acc')
# plt.title('Accuracy vs. Epoch')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Val Loss')
# plt.title('Loss vs. Epoch')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.tight_layout()
# plt.show()

# # Save Keras model
# # model.save('road_classifier_keras.h5')
# # print("Keras model saved as road_classifier_keras.h5")

# # Create TFLite converter WITHOUT quantization settings
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # No optimizations or quantization parameters
# # This will produce a standard float32 model without QUANTIZE/DEQUANTIZE ops

# # Convert model to TFLite
# tflite_model = converter.convert()

# # Save TFLite model
# with open('road_classifier_v3.tflite', 'wb') as f:
#     f.write(tflite_model)
# print("Float32 model saved as road_classifier_v3.tflite")

# print("\nNow convert to kmodel using:")
# print("ncc compile road_classifier_v3.tflite road_classifier.kmodel -i tflite -o kmodel -t k210 --dataset ./data/preprocessedv2")


Found 241 images belonging to 7 classes.
Found 58 images belonging to 7 classes.


C:\Users\23hangin\AppData\Local\Temp\ipykernel_30744\3335215770.py:48: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(


Epoch 1/5


C:\Users\23hangin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.3149 - loss: 1.9324
Epoch 1: val_accuracy improved from -inf to 0.65517, saving model to best_head_v3.h5


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.8765 - loss: 0.4537 - val_accuracy: 0.9310 - val_loss: 0.2263 - learning_rate: 1.0000e-04
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.8639 - loss: 0.3795
Epoch 8: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 8: val_accuracy did not improve from 0.93103
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.8643 - loss: 0.3787 - val_accuracy: 0.8793 - val_loss: 0.2761 - learning_rate: 1.0000e-04
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9191 - loss: 0.3198
Epoch 9: val_accuracy did not improve from 0.93103
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.9183 - loss: 0.3225 - val_accuracy: 0.8793 - val_loss: 0.2366 - learning_rate: 5.0000e-05
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9386 - loss: 0.3302
Epoch 10: val_accuracy improved from 0.93103 to 0.96552, saving model to best_finetune_v3.h5


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.9371 - loss: 0.3307 - val_accuracy: 0.9655 - val_loss: 0.1636 - learning_rate: 5.0000e-05
Epoch 11/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.8647 - loss: 0.3765
Epoch 11: val_accuracy did not improve from 0.96552
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.8663 - loss: 0.3734 - val_accuracy: 0.9138 - val_loss: 0.2054 - learning_rate: 5.0000e-05
Epoch 12/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9151 - loss: 0.2930
Epoch 12: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 12: val_accuracy did not improve from 0.96552
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.9162 - loss: 0.2889 - val_accuracy: 0.8966 - val_loss: 0.3330 - learning_rate: 5.0000e-05
Epoch 13/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9135 - loss: 0.2375
Epoch 13: val_accuracy did not improve from 0.96552
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.9145 - loss: 0.237

INFO:tensorflow:Assets written to: C:\Users\23hangin\AppData\Local\Temp\tmpsv56qe7i\assets


INFO:tensorflow:Assets written to: C:\Users\23hangin\AppData\Local\Temp\tmpsv56qe7i\assets


Saved artifact at 'C:\Users\23hangin\AppData\Local\Temp\tmpsv56qe7i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name='keras_tensor_159')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  1842887789264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887790992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887791376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887791184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887788880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887792528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887792912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887793296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887793104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1842887790416: TensorSpec(shape=(), dtype=tf.resource, name=None)


### **Get the correct order of labels for the maix end**

In [3]:
print(train_generator.class_indices)


{'forward': 0, 'intersection': 1, 'left_right_t': 2, 'left_turn': 3, 'right_turn': 4, 'straight_left': 5, 'straight_right': 6}


### **Small test with non-training images**

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = './data/testing/intersection-test.jpg'

#input_size = 224
input_size = 128

# Load and preprocess the image
img = image.load_img(img_path, target_size=(input_size, input_size))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize to match training preprocessing

# Predict
predictions = model.predict(img_array)

# Get class indices to map predictions back to labels
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}  # Invert mapping

# Get the predicted class
predicted_class_index = np.argmax(predictions, axis=1)[0]
predicted_label = class_labels[predicted_class_index]

print(f"Predicted road type: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted road type: intersection


### **Validate that the model is ready for conversion**

In [4]:
import flatbuffers
from tflite.Model import Model
import tflite.BuiltinOperator as BuiltinOp

# Build a mapping from code→name
op_names_1 = {
    val: name
    for name, val in BuiltinOp.__dict__.items()
    if isinstance(val, int)
}

# Load your TFLite flatbuffer
with open("road_classifier_v2.tflite", "rb") as f:
    buf = f.read()

model = Model.GetRootAsModel(buf, 0)

print("Ops used in v2:")
seen = set()
for i in range(model.OperatorCodesLength()):
    code = model.OperatorCodes(i).BuiltinCode()
    name = op_names_1.get(code, f"UNKNOWN({code})")
    if name not in seen:
        print(f"  • {name}")
        seen.add(name)

op_names_2 = {
    val: name
    for name, val in BuiltinOp.__dict__.items()
    if isinstance(val, int)
}

with open ("road_classifier_v3.tflite", "rb") as f:
    buf = f.read()

model = Model.GetRootAsModel(buf, 0)

print("Ops used v3:")
seen = set()
for i in range(model.OperatorCodesLength()):
    code = model.OperatorCodes(i).BuiltinCode()
    name = op_names_2.get(code, f"UNKNOWN({code})")
    if name not in seen:
        print(f"  • {name}")
        seen.add(name)



Ops used in v2:
  • CONV_2D
  • DEPTHWISE_CONV_2D
  • ADD
  • MEAN
  • FULLY_CONNECTED
  • SOFTMAX
Ops used v3:
  • CONV_2D
  • DEPTHWISE_CONV_2D
  • ADD
  • MEAN
  • FULLY_CONNECTED
  • SOFTMAX
